### Идея
Историю операций (для минимальной надежности) будем хранить в файле, а не оперативной памяти. \
Конечно, было бы лучше хранить историю транзакций в отдельном сервисе (к примеру, в реляционной БД), но для простоты в этом задании будем использовать файлы .xlsx. \
Они позволят хранить данные в табличном виде и удобно работать с ними в последствии.

### Что умеет решение ниже:
* Если изначально есть форматированный файл с историей, применять операции к стартовому балансу. Файл с историей хранит тип операции, сумму, ts операции.
* Если файла с историей нет - создавать пустой
* Логировать каждую операцию через логгер
* После каждой успешной операции записывать результат в файл с историей на диск для предотвращения потери данных
* Проверять введенные значения пополнения/снятия на адекватность - баланс не может уйти в минус, пополнение/снятие отрицательных значений невозможно, снятие строк со счета - тоже
* Все операции - в Decimal, т.к. работаем с балансом

In [1]:
from pathlib import Path
import logging
import polars as pl
import sys
from datetime import datetime
from decimal import Decimal

In [2]:
class Account():


    _DEFAULT_OPERATION_PATH = (Path(globals()['_dh'][0]) / 'op_history.xlsx').resolve()


    def __init__(
            self, 
            acc_name: str = 'hse_de_pyintro', 
            start_balance: Decimal = 0, 
            operation_history_path: Path | str | None = _DEFAULT_OPERATION_PATH
        ) -> None:
        """
        Инициализируем новый аккаунт

        Args:
            acc_name (str, optional): имя аккаунта. Defaults to 'hse_de_pyintro'.
            start_balance (float, optional): стартовый баланс. Defaults to 0.
            operation_history (Path, optional): путь к .xlsx файлу с историей операций. Defaults to (Path(__file__).parent / 'operation_histore.xlsx').resolve().
        """
        self.__logger = self.__set_logger()

        self.__acc_name = acc_name
        self.__balance = start_balance
        
        self.__operation_history = None
        self.__operation_history_path = self.__set_op_path(operation_history_path)
        self.__operation_history_ws_name = f"{self.__acc_name.upper()}_OP_HISTORY"
        self.__create_op_file_if_not_exists()
        self.__apply_history()
        

    def __set_logger(self) -> logging.Logger:
        """
        Создание логгера

        Returns:
            logging.Logger: логгер
        """     

        logging.basicConfig(stream=sys.stdout)
        logger = logging.getLogger('account_logger')
        logger.setLevel(logging.INFO)
        return logger
    

    def __set_op_path(self, op_path: Path | str) -> Path:
        """
        Проверяем, что в указанном пути корректное расширение

        Args:
            op_path (Path | str): предложенный путь

        Returns:
            Path: предложенный путь | дефолтный, если расширение указано не .xlsx
        """        
        op_path_suggested = Path(op_path).resolve()  
        op_path_extension = op_path.suffix

        # Если расширение предложенного пути не .xlsx, используем дефолтный путь
        if op_path_extension != '.xlsx':
            self.__logger.warning(
                "Предложенный путь %s имеет некорректное расширение.\n \
                Допускается только расширение .xlsx.\n \
                Путь с историей изменен на дефолтный - %s",
                op_path_suggested, self._DEFAULT_OPERATION_PATH
            )
            return self._DEFAULT_OPERATION_PATH
            
        return op_path_suggested
        

    def __create_op_file_if_not_exists(self) -> None:
        """
        Создаем файл по указанному пути, если его не существует
        """        
        
        if not self.__operation_history_path.is_file():        
           df = pl.DataFrame({"OP_NAME": [], "AMOUNT": [], "OP_DATE": []})
           df.write_excel(self.__operation_history_path, worksheet=self.__operation_history_ws_name)


    def __trigger_op_history_save(self):
        """
        Сохраняем историю операций на диск
        """        
        self.__operation_history.select(
            pl.col("OP_NAME"),
            pl.col("AMOUNT").cast(pl.String),
            pl.col("OP_DATE").cast(pl.String)
        ).write_excel(self.__operation_history_path, self.__operation_history_ws_name, autofit=True)


    def __apply_history(self):
        """
        Применяем историю операций к стартовому балансу
        """
        # Открываем первую страницу в .xlsx файле.
        # Если она называется не self.__operation_history_ws_name, сохраним ее копию в том же файле, с которой позже будем работать.
        # Предполагается, что есть заголовки в файле с историей
        df = pl.read_excel(self.__operation_history_path, raise_if_empty=False, engine='calamine') \
            .select(
            pl.col("OP_NAME"),
            pl.col("AMOUNT").cast(pl.Decimal(scale=4)),
            pl.col("OP_DATE").str.to_datetime("%Y-%m-%d %H:%M:%S%.f")
        )

        df_agg = df.group_by("OP_NAME").agg(pl.sum("AMOUNT"))

        df_topup = df_agg.filter(pl.col("OP_NAME") == 'TOPUP')
        df_topup_sum = df_topup.item(0, 1) if df_topup.select(pl.len()).item() > 0 else 0

        df_withdraw = df_agg.filter(pl.col("OP_NAME") == 'WITHDRAW')
        df_withdraw_sum = df_withdraw.item(0, 1) if df_withdraw.select(pl.len()).item() > 0 else 0

        self.__balance = self.__balance + df_topup_sum - df_withdraw_sum
        self.__operation_history = df
        self.__trigger_op_history_save()

        self.__logger.info(f"Применено {df.select(pl.len()).item()} операций на основе истории. Текущий баланс: {self.__balance}")


    def topup(self, topup_sum: float = 0):
        """
        Пополнение баланса

        Args:
            topup_sum (float, optional): сумма пополнения. Defaults to 0.
        """
        if not any(isinstance(topup_sum, typ) for typ in (float, int)) or topup_sum < 0:
            self.__logger.warning(
                "Введено некорректное или отрицательное число %s. Допускаются только положительные числа", topup_sum
            )
            return

        self.__balance += Decimal(str(topup_sum))
        df_topup = pl.DataFrame({"OP_NAME": ["TOPUP"], "AMOUNT": [topup_sum], "OP_DATE": [datetime.now()]}).select(
            pl.col("OP_NAME"),
            pl.col("AMOUNT").cast(pl.Decimal(scale=4)),
            pl.col("OP_DATE").cast(pl.Datetime)
        )
        self.__operation_history.extend(df_topup)
        self.__trigger_op_history_save()

        self.__logger.info(f"Добавлено {topup_sum} на баланс. Текущий баланс: {self.__balance}")


    def withdraw(self, withdraw_sum: float = 0):
        """
        Снятие с баланса

        Args:
            withdraw_sum (float, optional): сумма на снятие. Если на счете не хватает денег, логируем как предупреждение. Defaults to 0.
        """
        if not any(isinstance(withdraw_sum, typ) for typ in (float, int)) or withdraw_sum < 0:
            self.__logger.warning(
                "Введено некорректное или отрицательное число %s. Допускаются только положительные числа", withdraw_sum
            )
            return

        if self.__balance < withdraw_sum:
            self.__logger.info(f"Недостаточно средств для снятия {round(withdraw_sum, 2)}. Текущий баланс: {round(self.__balance, 2)}")
            return
        
        self.__balance -= Decimal(str(withdraw_sum))
        df_withdraw = pl.DataFrame({"OP_NAME": ["WITHDRAW"], "AMOUNT": [withdraw_sum], "OP_DATE": [datetime.now()]}).select(
            pl.col("OP_NAME"),
            pl.col("AMOUNT").cast(pl.Decimal(scale=4)),
            pl.col("OP_DATE").cast(pl.Datetime)
        )
        self.__operation_history.extend(df_withdraw)
        self.__trigger_op_history_save()

        self.__logger.info(f"Снято {withdraw_sum}. Текущий баланс: {self.__balance}")


    def get_balance(self) -> Decimal:
        """
        Получение ткущего баланса

        Returns:
            float: текущий баланс
        """        
        self.__logger.info(f"Текущий баланс: {self.__balance}")



In [3]:
my_acc = Account()

INFO:account_logger:Применено 0 операций на основе истории. Текущий баланс: 0


In [4]:
my_acc.topup(150.655)
my_acc.withdraw(10000.54678)
my_acc.withdraw(27.5)
my_acc.get_balance()
my_acc.topup(-1)
my_acc.topup(0)
my_acc.topup(1)
my_acc.topup(0.01)
my_acc.withdraw(123.165)
my_acc.topup('dbcde')
my_acc.get_balance()

INFO:account_logger:Добавлено 150.655 на баланс. Текущий баланс: 150.655
INFO:account_logger:Недостаточно средств для снятия 10000.55. Текущий баланс: 150.66
INFO:account_logger:Снято 27.5. Текущий баланс: 123.155
INFO:account_logger:Текущий баланс: 123.155
INFO:account_logger:Добавлено 0 на баланс. Текущий баланс: 123.155
INFO:account_logger:Добавлено 1 на баланс. Текущий баланс: 124.155
INFO:account_logger:Добавлено 0.01 на баланс. Текущий баланс: 124.165
INFO:account_logger:Снято 123.165. Текущий баланс: 1.000
INFO:account_logger:Текущий баланс: 1.000
